This function computes the interpolant for the hamiltonian of the Lagrangian vorticity barrier equation:

\begin{equation}
\mathrm{H}_{t_0}^{t_N}(\mathbf{x}_0) = \dfrac{\nu}{t_N-t_0} \delta \mathbf{\omega (\mathbf{x}_0, t_0,t_N)} \label{eq: vorticity} \tag{1},
\end{equation}

where $ \nu $ is the viscosity and $ \delta \hat{\omega} (\mathbf{x}_0, t_0,t_N) = \hat{\omega} (\mathbf{F}_{t_0}^{t_N}(\mathbf{x}_0), t_1) - \hat{\omega}(\mathbf{x}_0, t_0) $.
 
For simplicity we can normalize equation $ \ref{eq: vorticity} $ by $ \nu $ and thus simply omit them.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X_domain | array (Ny, Nx)| X-meshgrid of initial conditions |
| Y_domain | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 over which $ \hat{\omega}(\mathbf{x}, t) $ is defined |
| X | array (Ny, Nx)| X-meshgrid with Ny=Nx=1024 |
| Y | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 |
| omega | array (1024, 1025, 251) | $ \hat{\omega}(\mathbf{x}, t) $ |
| Fmap | array (Ny, Nx, N) | $ \mathbf{F}_{t_0}^{t}(\mathbf{x}_0) $, with $ t \in [t_0, t_1] $  |
| time | array (N, ) | $ t \in [t_0, t_1] $ |
| time_data | array | time of the velocity data |
| Interpolant_delta_omega |  RectBivariateSpline object | Interpolant for $ \mathrm{H}_{t_0}^{t_N} $|

In [ ]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-4])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

In [ ]:
# Import package for parallel computing
from joblib import Parallel, delayed

# Import package for progress bar
from tqdm.notebook import tqdm

# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# Import package for computing trajectories/velocity along trajectories
from ipynb.fs.defs.integration_dFdt import integration_dFdt

# Import numpy 
import numpy as np

In [3]:
def HamiltonianLagrangianActiveVorticity(X_domain, Y_domain, X, Y, omega, Fmap, time, time_data):
    '''
    Compute interpolant of right-hand-side of Lagrangian vorticity barrier equation.
    
    Parameters:
        X_domain:  array(Ny, Nx), X-meshgrid of domain
        Y_domain:  array(Ny, Nx), Y-meshgrid of domain
        X:         array(NY, NX), X-meshgrid of data domain
        Y:         array(NY, NX), Y-meshgrid of data domain
        omega:     array(NY, NX, NT), 3D meshgrid of vorticity over spatial and temporal data domain
        Fmap:      array(Nt, 2, Ny*Nx), Flow map of trajectories
        time:      array(Nt, ), time
        time_data: array(1,NT), time data
        
    Returns:
        Interpolant_delta_omega: Interpolant for Hamiltonian
    '''
    
    # iterate over all trajectories and compute omega along them
    
    # compute interpolant for omega over meshgrid X, Y over time
    Interpolant_omega = []
    # vorticity at t_0
    Interpolant_omega.append(RBS(Y[:,0], X[0,:], omega[:,:,0]))
    
    dt_data = time_data[0,1]-time_data[0,0]
    k = np.searchsorted(time_data[0,:], time[-1])
    
    if k >= time_data.shape[1]:
    
        Omega = (time_data[0,k+1]-time[-1])/dt_data*omega[:,:,k] + (time[-1]-time_data[0,k])/dt_data*omega[:,:,k+1]
        
    else:
        
        Omega = (time_data[0,k+1]-time[-1])/dt_data*omega[:,:,k] + (time[-1]-time_data[0,k])/dt_data*omega[:,:,k+1]
    
    # vorticity at t = tN (=time[-1])
    Interpolant_omega.append(RBS(Y[:,0], X[0,:], Omega))
    
    # evaluate delta_omega at at time tN 
    delta_omega = Interpolant_omega[-1](Fmap[-1,1,:].ravel(), Fmap[-1,0,:].ravel(), grid = False)-Interpolant_omega[0](Fmap[0,1,:].ravel(), Fmap[0,0,:].ravel(), grid = False)
    
    # Interpolant for delta_omega
    Interpolant_delta_omega = RBS(Y_domain[:,0], X_domain[0,:], delta_omega.reshape((X_domain.shape[0], Y_domain.shape[1])))
    
    return Interpolant_delta_omega